In [1]:
from sklearn import mixture
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
x1_txt = "../data/x1.txt"
x2_txt = "../data/x2.txt"
xeasy_txt = "../data/xeasy.txt"

In [3]:
x1 = np.genfromtxt(x1_txt, delimiter=',')
x2 = np.genfromtxt(x2_txt, delimiter=',')
xeasy = np.genfromtxt(xeasy_txt, delimiter=',')

In [4]:
K = [1, 2]

# Part 1

# xeasy

In [5]:
for k in K:
    gmm = mixture.GaussianMixture(init_params = "kmeans", max_iter = 100,  n_components = k)
    gmm.fit(xeasy)

    rd = 5
    print("k = ", k)
    print("pi_k: \n", np.round(gmm.weights_, rd))
    print("miu_k: \n", np.round(gmm.means_, rd))
    print("sigma_k: \n", np.round(gmm.covariances_, rd))
    print("")

k =  1
pi_k: 
 [1.]
miu_k: 
 [[1.79627 1.15065]]
sigma_k: 
 [[[ 3.17198 -2.27654]
  [-2.27654  3.49633]]]

k =  2
pi_k: 
 [0.59281 0.40719]
miu_k: 
 [[ 3.01465 -0.17291]
 [ 0.02246  3.07758]]
sigma_k: 
 [[[ 1.01065  0.15418]
  [ 0.15418  0.94718]]

 [[ 1.01104 -0.04961]
  [-0.04961  0.94407]]]



# X1

In [6]:
for k in K:
    gmm = mixture.GaussianMixture(init_params = "kmeans", max_iter = 100,  n_components = k)
    gmm.fit(x1)

    rd = 5
    print("k = ", k)
    print("pi_k: \n", np.round(gmm.weights_, rd))
    print("miu_k: \n", np.round(gmm.means_, rd))
    print("sigma_k: \n", np.round(gmm.covariances_, rd))
    print("") 

k =  1
pi_k: 
 [1.]
miu_k: 
 [[0.65826 1.33115]]
sigma_k: 
 [[[ 2.49394 -0.83207]
  [-0.83207  2.57821]]]

k =  2
pi_k: 
 [0.55437 0.44563]
miu_k: 
 [[-0.2869   2.17083]
 [ 1.83408  0.28655]]
sigma_k: 
 [[[ 1.58963  0.36036]
  [ 0.36036  2.07693]]

 [[ 1.12503 -0.09992]
  [-0.09992  1.23349]]]



# x2

In [7]:
for k in K:
    gmm = mixture.GaussianMixture(init_params = "kmeans", max_iter = 100,  n_components = k)
    gmm.fit(x2)

    rd = 5
    print("k = ", k)
    print("pi_k: \n", np.round(gmm.weights_, rd))
    print("miu_k: \n", np.round(gmm.means_, rd))
    print("sigma_k: \n", np.round(gmm.covariances_, rd))
    print("")

k =  1
pi_k: 
 [1.]
miu_k: 
 [[ 0.09421 -0.08491]]
sigma_k: 
 [[[5.06894 0.40546]
  [0.40546 5.05081]]]

k =  2
pi_k: 
 [0.5615 0.4385]
miu_k: 
 [[ 0.03484 -0.0399 ]
 [ 0.17023 -0.14255]]
sigma_k: 
 [[[ 1.31366  0.10374]
  [ 0.10374  0.97578]]

 [[ 9.86732  0.79962]
  [ 0.79962 10.26301]]]



# Part 2

In [9]:
from scipy.stats import multivariate_normal as mvn

def em(xs, pis, mus, sigmas, tol=0.01, max_iter=100):

    n, p = xs.shape
    k = len(pis)

    ll_old = 0
    for i in range(max_iter):
        exp_A = []
        exp_B = []
        ll_new = 0

        # E-step
        ws = np.zeros((k, n))
        for j in range(len(mus)):
            for i in range(n):
                ws[j, i] = pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
        ws /= ws.sum(0)

        # M-step
        pis = np.zeros(k)
        for j in range(len(mus)):
            for i in range(n):
                pis[j] += ws[j, i]
        pis /= n

        mus = np.zeros((k, p))
        for j in range(k):
            for i in range(n):
                mus[j] += ws[j, i] * xs[i]
            mus[j] /= ws[j, :].sum()

        sigmas = np.zeros((k, p, p))
        for j in range(k):
            for i in range(n):
                ys = np.reshape(xs[i]- mus[j], (2,1))
                sigmas[j] += ws[j, i] * np.dot(ys, ys.T)
            sigmas[j] /= ws[j,:].sum()

        # update complete log likelihoood
        ll_new = 0.0
        for i in range(n):
            s = 0
            for j in range(k):
                s += pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
            ll_new += np.log(s)

        if np.abs(ll_new - ll_old) < tol:
            break
        ll_old = ll_new

    return ll_new, pis, mus, sigmas

In [ ]:
def provableEM(x):

    
